In [1]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2

model = load_model("/home/jovyan/GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py", "/home/jovyan/GroundingDINO/weights/groundingdino_swint_ogc.pth").to("cuda")

final text_encoder_type: bert-base-uncased


In [2]:
import json
file_name = '/home/jovyan/Evaluation/q&a(gemma).json'
with open(file_name, 'r') as file:
    data = json.load(file)

In [3]:
correct = 0
objects = []

for i in range(len(data)):
    answer = data[i]['answer']
    
    IMAGE_PATH = "/home/jovyan/100.jpeg"
    TEXT_PROMPT = f"{answer}"
    
    BOX_TRESHOLD = 0.35
    TEXT_TRESHOLD = 0.25

    image_source, image = load_image(IMAGE_PATH)

    boxes, logits, phrases = predict(
        model=model,
        image=image,
        caption=TEXT_PROMPT,
        box_threshold=BOX_TRESHOLD,
        text_threshold=TEXT_TRESHOLD
    )

    annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
    cv2.imwrite("annotated_image.jpg", annotated_frame)
    answer = answer.lower()
    if answer in phrases:
        correct = correct + 1
        objects.append(answer)

error = (correct/len(data))*100
print(f" Model has a correct rate of {error}%")

 Model has a correct rate of 84.21052631578947%


In [4]:
print(objects)

['dog', 'flannel shirt', 'red leash', 'sand', 'ocean', 'dog collar', 'hand', 'red toy', 'paw', 'sky', 'beach', 'nothing', 'sandy', 'wavy', 'sun', 'golden']


In [ ]:
from PIL import Image
image = Image.open("annotated_image.jpg")
image